In [39]:
from urllib.request import urlopen # urlopen : url에 접속하기 위한 라이브러리, requests는 urllib를 사용하기 편하게 쓸 수 있게 만든 라이브러리
from bs4 import BeautifulSoup # BeautifulSoup : HTML태그를 함수를 이용해서 효과적으로 파싱할 수 있게 도와주는 라이브러리
from selenium import webdriver
import pandas as pd
import time

driver = webdriver.Chrome(r"/Users/noblyan/Desktop/chromedriver")
url="http://www.boonyang24.com/?sub=boonyang_info/info_list"
driver.get(url)

#delete_ad=driver.find_element_by_class_name("closeBtn")
#delete_ad.click()
time.sleep(3)
html=driver.page_source
soup=BeautifulSoup(html,'lxml')


boonyang = soup.find("div",{"class":"list_type_boonyang"})
#soup = 전체 페이지 불러옴
#boonyang은 그 중 테이블에 해당되는 곳 만 파싱
print(boonyang)
#필요한 정보 => 위치, 면적, 가격, 시기

ul_li = boonyang.find("ul", {"class":"top"})
ul_li.text
print(ul_li)
lis = ul_li.findAll("li") #ul안에 있는 li를 list에다가 넣어줌  [<li></li>....]
print(lis)
#for li in ul_li:  #ul안에 있는 li를 파싱
#    print(li)
lis[0].text

lis_col = []
lis_col.extend(lis[2])
lis_col.extend(lis[3])
lis_col.extend(lis[4])
lis_col.extend(lis[6])
#내가 필요한 열의 종류는 지역(사업명/위치), 규모(면적), 가격, 연도(분양/입주시기)) 
print(lis_col)
lis_col[0] = '지역명'
lis_col[1] = '규모구분'
lis_col[3] = '연도'
print(lis_col)

lis_col_frame = pd.DataFrame(lis_col)
lis_col_frame
#lis_col을 표로 나타낼 열의 정보로 가공하였다.

##################

Citycell = {}
sd = pd.DataFrame(Citycell)

w_city = soup.find_all('td', {'class':'subject'})

for i in range(0, 15):
    w_city[i]
    print(w_city[i].text)
print('\n')

for i in range(0, 15):
    w_city[i] = "서울"
    
del w_city[3]    #면적이 미정인 지역에 대해 리스트요소 제거.
del w_city[9]
print(w_city)
print('\n')
###################
w_size = soup.find_all('b')

for i in range(0, 12):
    del w_size[i]
    
w_size[0] = "전용면적 102㎡초과"
w_size[1] = "전용면적 60㎡이하"
w_size[2] = "전용면적 102㎡초과"
w_size[3] = "전용면적 60㎡초과 85㎡이하"
w_size[4] = "전용면적 85㎡초과 102㎡이하"
w_size[5] = "전용면적 102㎡초과"
w_size[6] = "전용면적 60㎡이하"
w_size[7] = "전용면적 102㎡초과"
w_size[8] = "전용면적 60㎡이하"
w_size[9] = "전용면적 60㎡이하"
w_size[10] = "전용면적 60㎡이하"
w_size[11] = "전용면적 102㎡초과"
w_size[12] = "전용면적 102㎡초과"

print(w_size)
####################
w_price = soup.find_all('p')
print(w_price)               #6번째 주기로 가격정보가 나오는걸 알 수 있다.
                            #'미정'처리된 두 지역은 빼고 생각하였다.
print('\n')
for i in range(1,16):
    print(w_price[6*i-1].text)    
print('\n')

w_price_l = []
#3번째, 9번째 항목을 없애줘야 한다.
for i in range(1,4):
    print(w_price[6*i-1].text)   
    w_price_l.extend(w_price[6*i-1])   #등차수열 이용.
for i in range(5,10):
    print(w_price[6*i-1].text)
    w_price_l.extend(w_price[6*i-1])
for i in range(11,16):
    print(w_price[6*i-1].text)
    w_price_l.extend(w_price[6*i-1])
print('\n')
print(w_price_l)
    
#################################
w_price_l[0] = 1947700
w_price_l[1] = 171960
w_price_l[2] = 1385000
w_price_l[3] = 85120
w_price_l[4] = 1289000
w_price_l[5] = 890000
w_price_l[6] = 345000
w_price_l[7] = 2277000
w_price_l[8] = 350500
w_price_l[9] = 240000
w_price_l[10] = 316200
w_price_l[11] = 661000
w_price_l[12] = 826400
print('\n')
print(w_price_l)
##################################
#평당분양가격
sd['분양가격'] = pd.to_numeric(w_price_l, errors = 'coerce')

w_square = []

for i in range(0, 13):
    w_square.append(w_price_l[i])
print(w_square)
w_square[0] = int(1947700/102)
w_square[1] = int(171960/60)
w_square[2] = int(1385000/102)
w_square[3] = int(85120/85)
w_square[4] = int(1289000/102)
w_square[5] = int(890000/102)
w_square[6] = int(345000/60)
w_square[7] = int(2277000/102)
w_square[8] = int(350500/60)
w_square[9] = int(240000/60)
w_square[10] = int(316200/60)
w_square[11] = int(661000/102)
w_square[12] = int(826400/102)
##################################
#연도에 대한 리스트 필요.
w_year = soup.find_all('p')
print(w_year)
#가격과 연도는 같은 <p>태그 안에 있다.
#분양가를 알아보는 프로젝트 이므로 입주일이 아닌 분양일 기준으로 리스트를 정리할 계획이다.
print('\n')
#7, 13, 19, 25 a+(n-1)d
#7+(i-1)*6        // 등차수열 이용.
w_year_l = []
for i in range(1,16):
    print(w_year[7+(i-1)*6].text)    
    w_year_l.extend(w_year[7+(i-1)*6])
    
    
    #extend(!@#$%^.text) 할때와 extend(#$%^&*()) 할때 차이가 나는이유???
print('\n')

print(w_year_l)
print('\n')

#'미정' 제거
del w_year_l[3]
del w_year_l[9]
print(w_year_l)
for i in range(0,6):
    w_year_l[i] = '2019'
for i in range(6,13):
    w_year_l[i] = '2018'
print('\n')
print(w_year_l)
Citycell['지역명'] = w_city
Citycell['규모구분'] = w_size
Citycell['가격'] = w_price_l
Citycell['연도'] = w_year_l
Citycell['분양가격'] = w_price_l
Citycell['평당분양가격'] = w_square


sd = pd.DataFrame(Citycell)
sd = sd[['지역명', '규모구분','분양가격','평당분양가격', '연도']]
sd
################################2번째 페이지###############################


##################################


ModuleNotFoundError: No module named 'selenium'

In [40]:
html ='http://www.boonyang24.com/?sub=boonyang_info/info_list&do1=&si1=&boonryu01=&boonryu02=&page={}&keyword=&tp=&cate=&area_code=&work_year=&hyun_cate='
soup = BeautifulSoup(html, 'html.parser') 
    
for n in range (4,11):
    url = html.format(n)
    webpage = urlopen(url)
    soup = BeautifulSoup(webpage, 'html.parser')
    boonyang = soup.find("div",{"class":"list_type_boonyang"})
    w_city3 = soup.find_all('td', {'class':'subject'})
    
    for i in range(0, 15):  
        w_city3[i]
        print(w_city3[i].text)
    for i in range(0, 15):
        w_city3[i] = "서울"
    for i in range(0, 15):
        w_city_l.append(w_city3[i])
    
print(w_city_l)
Citycell['지역명'] = w_city_l

sd = pd.DataFrame(Citycell)
sd = sd[['지역명']]
sd

DMC에코자이 서울특별시 서대문구 남가좌동 242-14 
아크로서울포레스트 서울특별시 성동구 성수동1가 685-700 (
한일노벨리아시티 서울특별시 성북구 장위동 8-2외 6필지
신길센트럴자이 서울특별시 영등포구 신길동 337-246
관악파크뷰 서울 관악구 봉천동 66번지 일대
M펠리체(빌라) 서울시 강서구 화곡동 424-80
위드하우스(빌라) 서울시 은평구 갈현동 489-9
서울항동지구중흥S-클래스 서울특별시 구로구 항동 100-5 (항동지
상계역센트럴푸르지오 서울특별시 노원구 상계동 85-33
상봉베스트원 서울특별시 중랑구 상봉동 101-18
오금공공주택지구2단지(공공분양) 서울특별시 송파구 오금동 99
H-House대림뉴스테이 서울특별시 영등포구 대림동 994-1
고덕센트럴IPARK 서울특별시 강동구 상일동 131번지 일
용산센트럴파크해링턴스퀘어 서울특별시 용산구 한강로3가 63-70번
인덕IPARK 서울특별시 노원구 월계동 633-31번지 
DMC롯데캐슬더퍼스트 서울시 은평구 수색동 361-10번지 일원
고덕센트럴푸르지오 서울특별시 강동구 고덕동 210-1
청명에버아트3(빌라) 서울특별시 동대문구 장안동 372-8
신정뉴타운아이파크위브 서울특별시 양천구 신월6동 551-1 일대
공덕솔리스타(오피스텔) 서울특별시 마포구 공덕동 30-19
공덕솔리스타(도·생) 서울특별시 마포구 공덕동 30-19
고덕롯데캐슬베네루체 서울특별시 강동구 상일동 187
파랑새 서울 광진구 자양동 683-6
여의휴젠느 서울특별시 영등포구 영등포동7가 94-9
세정앙떼빌 서울 은평구 불광천길 388-22
보라매SK뷰 서울 영등포구 신길동 1583-1
강남헤븐리치 서울 서초구 서초동 1339
목동 슬로우스퀘어 서울시 양천구 목동 406-28
비버리하임 서울시 서초구 양재동 5-9
메트로타운7차 서울시 관악구 신림동 신림동 1483-8
대치3차아이파크 서울특별시 강남구 대치동 945-20
동화아트빌 서울 서대문구 북가좌동 294-20
다윈팰리스 서울 송파구 문정동 25-3
신성9차 서울

,지역명
0,서울
1,서울
2,서울
3,서울
4,서울
5,서울
6,서울
7,서울
8,서울
9,서울


In [41]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_html(url) :
    html = ""
    resp = requests.get(url)
    if resp.status_code == 200 :
        html = resp.text
    return html

html = get_html('http://www.boonyang24.com/?sub=boonyang_info/info_list&do1=&si1=&boonryu01=&boonryu02=&page=4&keyword=&tp=&cate=&area_code=&work_year=&hyun_cate=')
#                    http://www.boonyang24.com/?sub=boonyang_info/info_list&do1=&si1=&boonryu01=&boonryu02=&page=5&keyword=&tp=&cate=&area_code=&work_year=&hyun_cate=
soup = BeautifulSoup(html, 'html.parser')
boonyang = soup.find("div",{"class":"list_type_boonyang"})
w_city3 = soup.find_all('td', {'class':'subject'})

Citycell = {}

w_city_l = []
for i in range(0, 75):  
    w_city3[i]
print(w_city3[i].text)
print('\n')

for i in range(0, 75):
    w_city3[i] = "서울"
for i in range(0, 75):
    w_city_l.append(w_city3[i])
    
print(w_city_l)
print('\n')


Citycell['지역명'] = w_city_l

sd = pd.DataFrame(Citycell)
sd = sd[['지역명']]

IndexError: list index out of range

In [42]:
html ='http://www.boonyang24.com/?sub=boonyang_info/info_list&do1=&si1=&boonryu01=&boonryu02=&page={}&keyword=&tp=&cate=&area_code=&work_year=&hyun_cate='
soup = BeautifulSoup(html, 'html.parser') 
    
for n in range (4,11):
    url = html.format(n)
    webpage = urlopen(url)
    soup = BeautifulSoup(webpage, 'html.parser')
    boonyang = soup.find("div",{"class":"list_type_boonyang"})
    w_city3 = soup.find_all('td', {'class':'subject'})
    
    for i in range(0, 15):  
        w_city3[i]
        print(w_city3[i].text)
    for i in range(0, 15):
        w_city3[i] = "서울"
    for i in range(0, 15):
        w_city_l.append(w_city3[i])
    
print(w_city_l)
Citycell['지역명'] = w_city_l

sd = pd.DataFrame(Citycell)
sd = sd[['지역명']]
sd

DMC에코자이 서울특별시 서대문구 남가좌동 242-14 
아크로서울포레스트 서울특별시 성동구 성수동1가 685-700 (
한일노벨리아시티 서울특별시 성북구 장위동 8-2외 6필지
신길센트럴자이 서울특별시 영등포구 신길동 337-246
관악파크뷰 서울 관악구 봉천동 66번지 일대
M펠리체(빌라) 서울시 강서구 화곡동 424-80
위드하우스(빌라) 서울시 은평구 갈현동 489-9
서울항동지구중흥S-클래스 서울특별시 구로구 항동 100-5 (항동지
상계역센트럴푸르지오 서울특별시 노원구 상계동 85-33
상봉베스트원 서울특별시 중랑구 상봉동 101-18
오금공공주택지구2단지(공공분양) 서울특별시 송파구 오금동 99
H-House대림뉴스테이 서울특별시 영등포구 대림동 994-1
고덕센트럴IPARK 서울특별시 강동구 상일동 131번지 일
용산센트럴파크해링턴스퀘어 서울특별시 용산구 한강로3가 63-70번
인덕IPARK 서울특별시 노원구 월계동 633-31번지 
DMC롯데캐슬더퍼스트 서울시 은평구 수색동 361-10번지 일원
고덕센트럴푸르지오 서울특별시 강동구 고덕동 210-1
청명에버아트3(빌라) 서울특별시 동대문구 장안동 372-8
신정뉴타운아이파크위브 서울특별시 양천구 신월6동 551-1 일대
공덕솔리스타(오피스텔) 서울특별시 마포구 공덕동 30-19
공덕솔리스타(도·생) 서울특별시 마포구 공덕동 30-19
고덕롯데캐슬베네루체 서울특별시 강동구 상일동 187
파랑새 서울 광진구 자양동 683-6
여의휴젠느 서울특별시 영등포구 영등포동7가 94-9
세정앙떼빌 서울 은평구 불광천길 388-22
보라매SK뷰 서울 영등포구 신길동 1583-1
강남헤븐리치 서울 서초구 서초동 1339
목동 슬로우스퀘어 서울시 양천구 목동 406-28
비버리하임 서울시 서초구 양재동 5-9
메트로타운7차 서울시 관악구 신림동 신림동 1483-8
대치3차아이파크 서울특별시 강남구 대치동 945-20
동화아트빌 서울 서대문구 북가좌동 294-20
다윈팰리스 서울 송파구 문정동 25-3
신성9차 서울

,지역명
0,서울
1,서울
2,서울
3,서울
4,서울
5,서울
6,서울
7,서울
8,서울
9,서울


In [35]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

base_url = 'http://movie.daum.net/moviedb/grade?movieId=2725&type=netizen&page={}'
 
for n in range (77) :
    url = base_url.format(n+1)
    webpage = urlopen(url)
    source = BeautifulSoup(webpage, 'html5lib')
    reviews = source.find_all('p', {'class':'desc_review'})
    
    for review in reviews :
        print(review.get_text().strip())


짱

딜런 커스만 오빠  머쮜다~~~~~~~~~~~~지금 모하세요?????ㅋ
로빈......그립습니다. 
오~ 캡틴~ 나의 캡틴~! 

지금 CGV 채널에서 방영 해주고 있어서 모든 일 제쳐두고 또  보고 있어요.
로빈 윌리엄스 그의 미소가 잊혀지지 않는다..카르페 디엠!Carpe Diem!
끝나고 마지막 장면만 열번은 돌려봤네요. 1991년 중2 때 담임선생님이 강추하셨던 영화를 마흔 넘어 이제야 처음 보고 눈물 흘리고 있습니다.  보고 싶습니다, 선생님.
10000점



명작은 평가하는게 아닙니다. 그냥 보고 느낍시다.진짜 명작 오브 명작 입니다.

내가 그때 뭘 안다고 이거보고 울었을까?  지금도 똑같이 운다.
Captain! My captain!
누구나 꼭 한번쯤은 봤으면 하는 영화~

최고!!!
내  인생 영화  내 인생 책
책으로 먼저 읽고 영화로도 봤는데! 그 때의 감동이 아직까지 느껴지네요!
오~캡틴 마이 캡틴

카르페디엠!  
보고싶은 선생님...



ㅠㅠ
카르페디엠

학창시절 중간고사를 보고 보게된 영화~마지막 엔딩이서 어깨를 들썩이며 여운에 잠겨 자리를 못 뜨던 영화 하얀 눈밭만 보면 쏘 뷰티풀이 생각나고 차안에 굴러다니는 작은 담요에 토드의 작문이 떠오르게 하던 영화 내인생을 이 영화를 보기전과 보고난후로 나눠놓은 영화
하고싶어요안될까요제발보내드립니다!
카르페디엠!
요즘은 왜 이런 영화가 안나와주는겨
당시 대학입시에 찌들어

힘들었던 우리들에게 많은 공감을 주었던  .

고등학교시절 극장에서보고 숨막힐정도의 감동을 받고 돌아왔는데 그땐 입시에 힘들때라 학교제도 입시에 대한 압박에 촛점을 두고 봤고
지금 어른이 되어 다시 보게된 영화속에서는
부모의 역할  그 아이가 뭘 하고 싶어하는지에대한 소통 
닐이 단 한마디 반항도 못한채 가슴에 묻고 죽음을 택할수밖에 없었던 그 마음이 너무도 절실하게 느껴져 많이 울고 가슴 아프게 봤습니다..



어릴때 이 영화를 볼수있었다면 뭔가 깨우치지않았을까 싶네요. 키팅선생님같은 선생님이 있을까.
나에게 

KeyboardInterrupt: 

In [50]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd


html ='http://www.boonyang24.com/?sub=boonyang_info/info_list&do1=&si1=&boonryu01=&boonryu02=&page={}&keyword=&tp=&cate=&area_code=&work_year=&hyun_cate='
soup = BeautifulSoup(html, 'html.parser') 

Citycell = {}

w_city_l = [] 
w_size_l = []
for n in range (4,6):
    url = html.format(n)
    webpage = urlopen(url)
    soup = BeautifulSoup(webpage, 'html.parser')
    boonyang = soup.find("div",{"class":"list_type_boonyang"})
    w_city3 = soup.find_all('td', {'class':'subject'})
    
    for i in range(0, 15):  
        w_city3[i]
        print(w_city3[i].text)
    for i in range(0, 15):
        w_city3[i] = "서울"
    for i in range(0, 15):
        w_city_l.append(w_city3[i])
        
        #####
    w_size2 = soup.find_all('b')
    for i in range(0, 15):
        del w_size2[i]
    #print(w_size2)

    for i in range(0, 15):
        w_size_l.append(w_size2[i])

    print(w_size_l)

Citycell['지역명'] = w_city_l
Citycell['규모구분'] = w_size_l
sd = pd.DataFrame(Citycell)

sd = sd[['지역명', '규모구분']]

####
'''
for n in range (10,15):
    url = html.format(n)
    webpage = urlopen(url)
    soup = BeautifulSoup(webpage, 'html.parser')
    boonyang = soup.find("div",{"class":"list_type_boonyang"})
    w_city3 = soup.find_all('td', {'class':'subject'})
    
    for i in range(0, 15):  
        w_city3[i]
        print(w_city3[i].text)
    for i in range(0, 15):
        w_city3[i] = "서울"
    for i in range(0, 15):
        w_city_l.append(w_city3[i])
print(w_city_l)
Citycell['지역명'] = w_city_l

sd = pd.DataFrame(Citycell)
sd = sd[['지역명']]
'''

DMC에코자이 서울특별시 서대문구 남가좌동 242-14 
아크로서울포레스트 서울특별시 성동구 성수동1가 685-700 (
한일노벨리아시티 서울특별시 성북구 장위동 8-2외 6필지
신길센트럴자이 서울특별시 영등포구 신길동 337-246
관악파크뷰 서울 관악구 봉천동 66번지 일대
M펠리체(빌라) 서울시 강서구 화곡동 424-80
위드하우스(빌라) 서울시 은평구 갈현동 489-9
서울항동지구중흥S-클래스 서울특별시 구로구 항동 100-5 (항동지
상계역센트럴푸르지오 서울특별시 노원구 상계동 85-33
상봉베스트원 서울특별시 중랑구 상봉동 101-18
오금공공주택지구2단지(공공분양) 서울특별시 송파구 오금동 99
H-House대림뉴스테이 서울특별시 영등포구 대림동 994-1
고덕센트럴IPARK 서울특별시 강동구 상일동 131번지 일
용산센트럴파크해링턴스퀘어 서울특별시 용산구 한강로3가 63-70번
인덕IPARK 서울특별시 노원구 월계동 633-31번지 
[<b title="48 평">161</b>, <b title="104 평">344</b>, <b title="8 평">27</b>, <b title="51 평">170</b>, <b title="34 평">114</b>, <b title="15 평">49</b>, <b title="10 평">35</b>, <b title="59 평">197</b>, <b title="33 평">110</b>, <b title="12 평">42</b>, <b title="37 평">124</b>, <b title="17 평">58</b>, <b title="42 평">139</b>, <b title="57 평">188</b>, <b title="35 평">117</b>]
DMC롯데캐슬더퍼스트 서울시 은평구 수색동 361-10번지 일원
고덕센트럴푸르지오 서울특별시 강동구 고덕동 210-1
청명에버아트3(빌라) 서울특별시 동대문구 장안동 372-8
신정뉴타운아이파크위브 서울특별시 양천구 신월6동 551-1 일대
공덕솔리스타(오피스텔) 

IndexError: list assignment index out of range

In [17]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd


html ='http://www.boonyang24.com/?sub=boonyang_info/info_list&do1=&si1=&boonryu01=&boonryu02=&page={}&keyword=&tp=&cate=&area_code=&work_year=&hyun_cate='
soup = BeautifulSoup(html, 'html.parser') 

Citycell = {}

w_city_l = [] 
w_year_l = []
for n in range (4,9):
    url = html.format(n)
    webpage = urlopen(url)
    soup = BeautifulSoup(webpage, 'html.parser')
    boonyang = soup.find("div",{"class":"list_type_boonyang"})
    w_city3 = soup.find_all('td', {'class':'subject'})
    w_year2 = soup.find_all('p')
    
    for i in range(0, 15):  
        w_city3[i]
        print(w_city3[i].text)
    for i in range(0, 15):
        w_city3[i] = "서울"
    for i in range(0, 15):
        w_city_l.append(w_city3[i])
print(w_city_l)

#지금까지 총 인덱스 0~42
for n in range (4,9):
    url = html.format(n)
    webpage = urlopen(url)
    soup = BeautifulSoup(webpage, 'html.parser')
    boonyang = soup.find("div",{"class":"list_type_boonyang"})
    w_year3 = soup.find_all('p')

    for i in range(1,16):
        print(w_year3[7+(i-1)*6].text)    
        w_year_l.extend(w_year3[7+(i-1)*6])

for i in range(0,75):
    w_year_l[i] = '2017'
print('\n')
print(w_year_l)
    
Citycell['지역명'] = w_city_l
Citycell['연도'] = w_year_l
sd = pd.DataFrame(Citycell)
sd = sd[['지역명', '연도']]
sd

이수역리가 서울특별시 동작구 사당동 1157
DMC에코자이 서울특별시 서대문구 남가좌동 242-14 
아크로서울포레스트 서울특별시 성동구 성수동1가 685-700 (
한일노벨리아시티 서울특별시 성북구 장위동 8-2외 6필지
신길센트럴자이 서울특별시 영등포구 신길동 337-246
관악파크뷰 서울 관악구 봉천동 66번지 일대
M펠리체(빌라) 서울시 강서구 화곡동 424-80
위드하우스(빌라) 서울시 은평구 갈현동 489-9
서울항동지구중흥S-클래스 서울특별시 구로구 항동 100-5 (항동지
상계역센트럴푸르지오 서울특별시 노원구 상계동 85-33
상봉베스트원 서울특별시 중랑구 상봉동 101-18
오금공공주택지구2단지(공공분양) 서울특별시 송파구 오금동 99
H-House대림뉴스테이 서울특별시 영등포구 대림동 994-1
고덕센트럴IPARK 서울특별시 강동구 상일동 131번지 일
용산센트럴파크해링턴스퀘어 서울특별시 용산구 한강로3가 63-70번
인덕IPARK 서울특별시 노원구 월계동 633-31번지 
DMC롯데캐슬더퍼스트 서울시 은평구 수색동 361-10번지 일원
고덕센트럴푸르지오 서울특별시 강동구 고덕동 210-1
청명에버아트3(빌라) 서울특별시 동대문구 장안동 372-8
신정뉴타운아이파크위브 서울특별시 양천구 신월6동 551-1 일대
공덕솔리스타(오피스텔) 서울특별시 마포구 공덕동 30-19
공덕솔리스타(도·생) 서울특별시 마포구 공덕동 30-19
고덕롯데캐슬베네루체 서울특별시 강동구 상일동 187
파랑새 서울 광진구 자양동 683-6
여의휴젠느 서울특별시 영등포구 영등포동7가 94-9
세정앙떼빌 서울 은평구 불광천길 388-22
보라매SK뷰 서울 영등포구 신길동 1583-1
강남헤븐리치 서울 서초구 서초동 1339
목동 슬로우스퀘어 서울시 양천구 목동 406-28
비버리하임 서울시 서초구 양재동 5-9
메트로타운7차 서울시 관악구 신림동 신림동 1483-8
대치3차아이파크 서울특별시 강남구 대치동 945-20
동화아트빌 서울 서대문구 북가좌동 294-20
다윈팰리

,지역명,연도
0,서울,2017
1,서울,2017
2,서울,2017
3,서울,2017
4,서울,2017
5,서울,2017
6,서울,2017
7,서울,2017
8,서울,2017
9,서울,2017
